# CV ile En İyi Hiperparametreler?
* Her zaman best_params'ı mı almalıyız?

* Bu notebookta klasik bir seçim olan RandomSearchCV incelenmiştir. Optuna gibi daha sofistik yöntemler daha sonra incelenecektir.

## Hazırlayan: https://github.com/Frightera

In [ ]:
import numpy as np

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer

from scipy.stats import t as t_dist

In [ ]:
# 5000 örnek ve 30 feature'dan oluşan regresyon veriseti
X, y = make_regression(n_samples=5000, n_features=30, random_state=42)

# %80'i train, %20'si test olacak şekilde split edelim.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CV objesini ayarlayalım.
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# RandomForest modeli
model = RandomForestRegressor(random_state=42)

# Bazı parametre aralıklarını belirleyelim.
param_grid = {
    'max_depth': np.arange(1, 21),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 11),
}

# CV kullanarak RandomSearch yap.
random_search = RandomizedSearchCV(
    model,
    n_iter=15,
    param_distributions=param_grid,
    cv=cv,
    scoring=make_scorer(mean_squared_error, greater_is_better=False),
    n_jobs=-1,
    verbose=1
)

# Tuning işlemini başlat
random_search.fit(X_train, y_train)

# En iyi parametre ve skorları al.
best_params = random_search.best_params_
best_score = -random_search.best_score_

print(f"Best parameters: {best_params}")
# {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 12}

print(f"Best mean squared error: {best_score}") # 5856.025

# En iyi modeli test datasında dene
best_model = random_search.best_estimator_
test_preds = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds)

print(f"Test MSE: {test_mse}") # 5555.029

In [72]:
# mean_test_score'a göre CV skorlarını küçükten büyüğe sırala
sorted_results = sorted(zip(cv_results['params'], cv_results['mean_test_score'], 
                            cv_results['std_test_score']), key=lambda x: x[1], reverse=True)

# --> Neden t-distribution kullandık?
# Küçük bir örneklem boyutuyla (çapraz doğrulamadaki kat sayısı) çalıştığımız ve 
# gerçek popülasyon standart sapmasını bilmediğimiz için t-dağılımı kullanıldı. 
# Küçük örneklem büyüklükleri ve bilinmeyen popülasyon parametreleri ile çalışırken, 
# t-dağılımı, güven aralıkları için normal (Gauss) dağılımdan daha iyi bir yaklaşım sağlar.
# Bizim durumumuzda, çapraz doğrulamada 5 kat ile çalıştığımız için, örneklem boyutu küçüktür ve 
# ortalama karesel hatanın %95 güven aralığını hesaplamak için t dağılımını kullanmak daha uygundur.

# --> Burada gerçek popülasyondan kastın nedir?
# Bu durumda, "gerçek popülasyon", modeli tüm olası eğitim ve doğrulama kümeleri 
# kombinasyonlarında değerlendirebilseydik elde edilebilecek olası model performans 
# ölçümlerinin (örneğin, ortalama karesel hata) tam kümesini ifade eder. Bu pratik 
# olarak imkansız olduğundan, modelin görünmeyen veriler üzerindeki performansını 
# tahmin etmek için bir yaklaşım yöntemi olarak çapraz doğrulama kullanıyoruz.
alpha = 0.05
n_splits = cv.get_n_splits()
critical_value = t_dist.ppf(1 - alpha / 2, n_splits - 1)

for params, mean_mse, std_mse in sorted_results:
    # Standard error'u hesapla
    sem = std_mse / np.sqrt(n_splits)

    # %95'lik confidence interval hesapla
    lower_bound = -mean_mse - critical_value * sem
    upper_bound = -mean_mse + critical_value * sem

    print(f"Params: {params}")
    print(f"  Mean MSE: {-mean_mse}")
    print(f"  Std MSE: {std_mse}")
    print(f"  95% Confidence Interval: [{lower_bound}, {upper_bound}]\n")

Params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 12}
  Mean MSE: 5856.025745117241
  Std MSE: 195.61486368213548
  95% Confidence Interval: [5613.1378113695955, 6098.913678864887]

Params: {'min_samples_split': 7, 'min_samples_leaf': 1, 'max_depth': 17}
  Mean MSE: 5858.122492576592
  Std MSE: 189.81267439392056
  95% Confidence Interval: [5622.438928378887, 6093.806056774297]

Params: {'min_samples_split': 9, 'min_samples_leaf': 2, 'max_depth': 19}
  Mean MSE: 5873.073266837842
  Std MSE: 202.64119093200762
  95% Confidence Interval: [5621.4609955044325, 6124.685538171252]

Params: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 11}
  Mean MSE: 5936.989487866667
  Std MSE: 181.7739650235231
  95% Confidence Interval: [5711.287299686208, 6162.691676047127]

Params: {'min_samples_split': 9, 'min_samples_leaf': 4, 'max_depth': 17}
  Mean MSE: 5949.826554113563
  Std MSE: 193.03137600342473
  95% Confidence Interval: [5710.146444006376, 6189.50666422075]



## Bazı Parametrelerle Modelleri Yeniden Fit Edelim

### En iyi parametreler

In [74]:
params = {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 12}

model = RandomForestRegressor(random_state=42, **params)

model.fit(X_train, y_train)

test_preds = model.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds)
train_mse = mean_squared_error(y_train, model.predict(X_train))

print(f"Test MSE: {test_mse}")
print(f"Train MSE: {train_mse}")

Test MSE: 5555.0299165158785
Train MSE: 878.3674882489294


### En iyi 2.parametreler

In [73]:
params = {'min_samples_split': 7, 'min_samples_leaf': 1, 'max_depth': 17}

model = RandomForestRegressor(random_state=42, **params)

model.fit(X_train, y_train)

test_preds = model.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds)
train_mse = mean_squared_error(y_train, model.predict(X_train))

print(f"Test MSE: {test_mse}")
print(f"Train MSE: {train_mse}")

Test MSE: 5510.2412353180825
Train MSE: 1055.2659972931892


### Std en düşük parametreler

In [66]:
params = {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 11}

model = RandomForestRegressor(random_state=42, **params)

model.fit(X_train, y_train)

test_preds = model.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds)
train_mse = mean_squared_error(y_train, model.predict(X_train))

print(f"Test MSE: {test_mse}")
print(f"Train MSE: {train_mse}")

Test MSE: 5646.224568847524
Train MSE: 1142.5695931930002


## Yorumlar

### En iyi parametreleri uygularken, farklı farklı kriterlere bakmak önemli olabilir.
 * #### CV Foldlarındaki ortalama değer
 * #### CV Foldlarındaki skorun standart sapması
 

#### Genel Açıklama
Pratikte veya gerçek hayat uygulamarında, en iyi modeli seçerken yalnızca ortalama performansı değil aynı zamanda performanstaki değişkenliği (örn. standart sapma) ve model karmaşıklığı, eğitim süresi ve yorumlanabilirlik gibi diğer faktörleri de dikkate almak önemlidir. 

Şuanda, cross-validation uyguladığımız çalışmada daha düşük standart sapmaya sahip ikinci en iyi parametreler foldlar arasında daha tutarlı performans ve test veri setine daha iyi genelleme gösterdiği için daha uygun bir seçim olabilir.